In [3]:

import numpy as np

In [ ]:
pip install pandas tensorflow


In [9]:
pip install --upgrade

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\ASUS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
import pandas as pd
from IPython.display import clear_output
clear_output()



In [ ]:
pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-0.12.0-py3-none-any.whl

In [11]:
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\pywrap_tensorflow.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow', [dirname(__file__)])
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\imp.py", line 297, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\__init__.py", line 54, in <module>

In [ ]:
pip install --upgrade pip

In [13]:
pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tenssorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tenssorflow (from versions: none)
ERROR: No matching distribution found for tenssorflow


In [ ]:
import tensorflow as tf

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# Load data from CSV
df = pd.read_csv('continuous dataset.csv')

In [ ]:
#************************************** Data Pre-Processing ****************************************

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator

class TabularLSTMDataPreprocessor:
    def __init__(self, dataframe, target_column, time_column, categorical_columns=None,
                 scaler='minmax', sequence_length=24, batch_size=32, random_state=None):
        self.dataframe = dataframe
        self.target_column = target_column
        self.time_column = time_column
        self.categorical_columns = categorical_columns if categorical_columns else []
        self.scaler = self.get_scaler(scaler)
        self.sequence_length = sequence_length
        self.batch_size = batch_size
        self.random_state = random_state

    def get_scaler(self, scaler_type):
        if scaler_type == 'minmax':
            return MinMaxScaler()
        elif scaler_type == 'standard':
            return StandardScaler()
        else:
            raise ValueError("Invalid scaler type. Use 'minmax' or 'standard'.")

    def preprocess(self):
        #Creating lag features
        windows = [12, 24, 128]
        for column in self.dataframe.columns:
            if column != self.time_column and column not in self.categorical_columns:
                for window in windows:
                    self.dataframe[f"{column}_lag_{window}"] = self.dataframe[column].shift(window)
                    # Add other transformations as needed

        # Drop rows with missing values
        self.dataframe.dropna(inplace=True)

        # Sort by time
        self.dataframe.sort_values(by=[self.time_column], inplace=True)
        
        # Scale numerical features
        numerical_columns = [col for col in self.dataframe.columns if col not in [self.target_column, self.time_column, self.categorical_columns]]
        self.dataframe[numerical_columns] = self.scaler.fit_transform(self.dataframe[numerical_columns])

        # Apply one-hot encoding to categorical columns (if any)
        if self.categorical_columns:
            self.dataframe = pd.get_dummies(self.dataframe, columns=self.categorical_columns, drop_first=True)
        
        train_df = self.dataframe[self.dataframe[self.time_column] < '2019-01-01']
        test_df = self.dataframe[self.dataframe[self.time_column] >= '2019-01-01']
        # Split data into train and test sets
        X_train = train_df.drop(columns=[self.target_column, self.time_column]).values.astype(np.float32)
        y_train = train_df[self.target_column].values.astype(np.float32)
#         X = X.astype(np.float32)
#         y = y.astype(np.float32)
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size, random_state=self.random_state, shuffle=False)
        X_test = test_df.drop(columns=[self.target_column, self.time_column]).values.astype(np.float32)
        y_test = test_df[self.target_column].values.astype(np.float32)
        
        # Create time series generators for training and testing
        train_data_gen = TimeseriesGenerator(X_train, y_train,
                                             length=self.sequence_length, batch_size=self.batch_size)
        test_data_gen = TimeseriesGenerator(X_test, y_test,
                                            length=self.sequence_length, batch_size=self.batch_size)
    

        return train_data_gen, test_data_gen 